In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%config InlineBackend.figure_format = 'svg'

# Introduction

In this notebook, we collect the data needed for this case. We draw on two primary sources:

1. Federal Reserve Economic Data (FRED): The St. Louis Fed maintains FRED, a repository that aggregates data from many sources (such as the Bureau of Economic Data, the Bureau of Labor Statistics, the White House Office of Management and Budget). We use this source for our economic data.
2. Wikipedia and basically elementary school history of politics for political data.

# Economic Data

While the economic data exists in FRED, an easy way to retrieve it is to use the `pandas_datareader` package that interacts with FRED to fetch data in dataframes. Thanks to the good folks at AQR who open sourced `pandas` and `pandas_datareader`, we have very convenient ways of fetching and displaying this data.

In [2]:
import pandas
from pandas_datareader import data, wb
import datetime

We are interested in the data from 1929 to 2015. We set the start date to 1929 because most of the data sources on FRED is unavailable prior to 1929.

In [3]:
START_DATE = datetime.datetime(1929, 1, 1)
END_DATE = datetime.datetime(2016, 1, 1)

Each data series in FRED is specified by a symbol. For example, the annual real GDP is represented by the symbol `GDPCA` and has the URL [https://research.stlouisfed.org/fred2/series/GDPCA](https://research.stlouisfed.org/fred2/series/GDPCA). The series that we are interested in are:

- `gdp_r`: [GDPCA](https://research.stlouisfed.org/fred2/series/GDPCA) Real Gross Domestic Product, Billions of Chained 2009 Dollars, Annual, Not Seasonally Adjusted 1929 to 2015
- `gdp_n`: [GDPA](https://research.stlouisfed.org/fred2/series/GDPA) Gross Domestic Product, Billions of Dollars, Annual, Not Seasonally Adjusted 1929 to 2015
- `cpi`: [CPIAUCNS](https://research.stlouisfed.org/fred2/series/CPIAUCSL) Consumer Price Index for All Urban Consumers: All Items, Monthly, Not Seasonally Adjusted 1913-01 to 2016-04
- `fed_sur`: [FYFSD](https://research.stlouisfed.org/fred2/series/FYFSD) Federal Surplus or Deficit [-] Millions of Dollars, Annual, Not Seasonally Adjusted (FYFSD = FYFR - FYONET)
- `fed_rec`: [FYFR](https://research.stlouisfed.org/fred2/series/FYFR) Federal Receipts, Millions of Dollars, Annual, Not Seasonally Adjusted 1901 to 2015
- `fed_out`: [FYONET](https://research.stlouisfed.org/fred2/series/FYONET) Federal Net Outlays, Millions of Dollars, Annual, Not Seasonally Adjusted 1901 to 2015
- `def_pct_gdp`: [A824RE1A156NBEA](https://research.stlouisfed.org/fred2/series/A824RE1A156NBEA) Shares of gross domestic product: Government consumption expenditures and gross investment: Federal: National defense, Annual, Not Seasonally Adjusted, 1929 to 2015

We want to rename the series symbols (which are the default column headers in the dataframe) to something more readable. For example, we rename `GDPCA` to `gdp_r` for real GDP.

We can now fetch the data from FRED using `pandas_datareader` and rename the columns accordingly.

In [4]:
fred_data = data.DataReader(name=['GDPCA', 'GDPA', 'CPIAUCNS', 'FYFSD', 'FYFR', 'FYONET', 'A824RE1A156NBEA'], data_source='fred', start=START_DATE, end=END_DATE)
fred_data = fred_data.resample('AS').first().dropna()
fred_data.columns = ['gdp_r', 'gdp_n', 'cpi', 'fed_sur', 'fed_rec', 'fed_out', 'def_pct_gdp']

Then, we take a quick look at the kind of data we fetched:

In [5]:
fred_data

,gdp_r,gdp_n,cpi,fed_sur,fed_rec,fed_out,def_pct_gdp
DATE,,,,,,,
1929-01-01,1056.6,104.6,17.100,734.0,3862.0,3127.0,1.0
1930-01-01,966.7,92.2,17.100,738.0,4058.0,3320.0,1.2
1931-01-01,904.8,77.4,15.900,-462.0,3116.0,3577.0,1.4
1932-01-01,788.2,59.5,14.300,-2735.0,1924.0,4659.0,1.7
1933-01-01,778.3,57.2,12.900,-2602.0,1997.0,4598.0,1.7
1934-01-01,862.2,66.8,13.200,-3586.0,2955.0,6541.0,1.4
1935-01-01,939.0,74.3,13.600,-2803.0,3609.0,6412.0,1.6
1936-01-01,1060.5,84.9,13.800,-4304.0,3923.0,8228.0,1.6
1937-01-01,1114.6,93.0,14.100,-2193.0,5387.0,7580.0,1.5


In [6]:
fred_data['cpi_inflation'] = fred_data['cpi'].pct_change()
fred_data['gdp_r_growth'] = fred_data['gdp_r'].pct_change()
fred_data['gdp_n_growth'] = fred_data['gdp_n'].pct_change()
# first row becomes NA for percent change because there's no 0th value to percent change on
# so we drop rows with na. i.e. the row for 1929
fred_data = fred_data.dropna()

In [7]:
fred_data

,gdp_r,gdp_n,cpi,fed_sur,fed_rec,fed_out,def_pct_gdp,cpi_inflation,gdp_r_growth,gdp_n_growth
DATE,,,,,,,,,,
1930-01-01,966.7,92.2,17.100,738.0,4058.0,3320.0,1.2,0.000000,-0.085084,-0.118547
1931-01-01,904.8,77.4,15.900,-462.0,3116.0,3577.0,1.4,-0.070175,-0.064032,-0.160521
1932-01-01,788.2,59.5,14.300,-2735.0,1924.0,4659.0,1.7,-0.100629,-0.128868,-0.231266
1933-01-01,778.3,57.2,12.900,-2602.0,1997.0,4598.0,1.7,-0.097902,-0.012560,-0.038655
1934-01-01,862.2,66.8,13.200,-3586.0,2955.0,6541.0,1.4,0.023256,0.107799,0.167832
1935-01-01,939.0,74.3,13.600,-2803.0,3609.0,6412.0,1.6,0.030303,0.089074,0.112275
1936-01-01,1060.5,84.9,13.800,-4304.0,3923.0,8228.0,1.6,0.014706,0.129393,0.142665
1937-01-01,1114.6,93.0,14.100,-2193.0,5387.0,7580.0,1.5,0.021739,0.051014,0.095406
1938-01-01,1077.7,87.4,14.200,-89.0,6751.0,6840.0,1.7,0.007092,-0.033106,-0.060215


Looks neat eh?

# Political Data

Now we are interested in the political party of the president, senate, and the house for every year from 1929 to 2015

The data below is collected from [http://www.infoplease.com/ipa/A0774721.html](http://www.infoplease.com/ipa/A0774721.html). The data represents the state of the political system as of Jan 1 of that year. Hence, for an election year like 2012, the data shows the presidency as Republican (Bush) since Obama was not elected till September, and we show the presidency as of Jan 1.

- `dem_pres`: Binary variable with 0 for Republican president and 1 for Democrat president for a given year
- `total_senate`: Total number of senators in Congress for a given year
- `dem_senate`: Number of Democrat senators for a given year
- `rep_senate`: Number of Republican senators for a given year
- `total_house`: Total number of representatives in Congress for a given year
- `dem_house`: Number of Democrat representatives in Congress for a given year
- `rep_house`: Number of Republican representatives in Congress for a given year

We manually collected the data from the URL above. Unfortunately, there's no cool way to do this programmatically. We import this data from a the CSV file `president_congress.csv`.

In [8]:
president_congress = pandas.read_csv('president_congress.csv', index_col='date', parse_dates=True)

And we take a quick look at this data as well.

In [9]:
president_congress

,dem_pres,total_senate,dem_senate,rep_senate,total_house,dem_house,rep_house
date,,,,,,,
1929-01-01,0,96,47,48,435,195,237
1930-01-01,0,96,39,56,435,163,267
1931-01-01,0,96,39,56,435,163,267
1932-01-01,0,96,47,48,435,217,217
1933-01-01,1,96,47,48,435,217,217
1934-01-01,1,96,59,36,435,313,117
1935-01-01,1,96,59,36,435,313,117
1936-01-01,1,96,69,25,435,322,103
1937-01-01,1,96,69,25,435,322,103


Also pretty neat.

# Merging and Storing

Now we merge both sets of data (since they share a common date index) and save it into a single CSV file.

In [10]:
all_data = president_congress.merge(fred_data, left_index=True, right_index=True)
all_data

,dem_pres,total_senate,dem_senate,rep_senate,total_house,dem_house,rep_house,gdp_r,gdp_n,cpi,fed_sur,fed_rec,fed_out,def_pct_gdp,cpi_inflation,gdp_r_growth,gdp_n_growth
1930-01-01,0,96,39,56,435,163,267,966.7,92.2,17.100,738.0,4058.0,3320.0,1.2,0.000000,-0.085084,-0.118547
1931-01-01,0,96,39,56,435,163,267,904.8,77.4,15.900,-462.0,3116.0,3577.0,1.4,-0.070175,-0.064032,-0.160521
1932-01-01,0,96,47,48,435,217,217,788.2,59.5,14.300,-2735.0,1924.0,4659.0,1.7,-0.100629,-0.128868,-0.231266
1933-01-01,1,96,47,48,435,217,217,778.3,57.2,12.900,-2602.0,1997.0,4598.0,1.7,-0.097902,-0.012560,-0.038655
1934-01-01,1,96,59,36,435,313,117,862.2,66.8,13.200,-3586.0,2955.0,6541.0,1.4,0.023256,0.107799,0.167832
1935-01-01,1,96,59,36,435,313,117,939.0,74.3,13.600,-2803.0,3609.0,6412.0,1.6,0.030303,0.089074,0.112275
1936-01-01,1,96,69,25,435,322,103,1060.5,84.9,13.800,-4304.0,3923.0,8228.0,1.6,0.014706,0.129393,0.142665
1937-01-01,1,96,69,25,435,322,103,1114.6,93.0,14.100,-2193.0,5387.0,7580.0,1.5,0.021739,0.051014,0.095406
1938-01-01,1,96,75,17,435,333,89,1077.7,87.4,14.200,-89.0,6751.0,6840.0,1.7,0.007092,-0.033106,-0.060215
1939-01-01,1,96,75,17,435,333,89,1163.6,93.5,14.000,-2846.0,6295.0,9141.0,1.8,-0.014085,0.079707,0.069794


In [11]:
all_data.to_csv('all_data.csv')